# 25. 계산 그래프 시각화(1)

- 

## 25.1 Graphviz 설치하기

- 

## 25.2 DOT 언어로 그래프 작성하기

- 

## 25.3 노드에 속성 지정하기

- 

## 25.4 노드 연결하기

- 

# 26. 계산 그래프 시각화(2)

- 

## 26.1 시각화 코드 예

- 

## 26.2 계산 그래프에서 DOT 언어로 변환하기

- 

## 26.3 이미지 변환까리 한 번에

- 

## 26.4 동작 확인

- 

# 27. 테일러 급수 미분

- 

## 27.1 sin 함수 구현

- 

## 27.2 테일러 급수 이론

- 

## 27.3 테일러 급수 구현

- 

## 27.4 계산 그래프 시각화

- 

# 28. 함수 최적화

- 

## 28.1 로젝브록 함수

- 

## 28.2 미분 계산하기

- 

## 28.3 경사하강법 구현

- 

# 29. 뉴턴 방법으로 푸는 최적화(수동 계산)

- 

## 29.1 뉴턴 방법을 활용한 최적화 이론

- 

## 29.2 뉴턴 방법을 활용한 최적화 구현

- 

# 30. 고차 미분(준비 편)

- 

## 30.1 확인 (1): Variable 인스턴스 변수

-  

## 30.2 확인 (2): Function 클래스

-  

## 30.3 확인 (3): Variable 클래스의 역전파

-  